In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import os
import shutil

In [ ]:
base_dir = './cat_dog'
train_dir = base_dir + '/train'
train_dog_dir = train_dir + '/dog'
train_cat_dir = train_dir + '/cat'
test_dir = base_dir + '/test'
test_dog_dir = test_dir + '/dog'
test_cat_dir = test_dir + '/cat'
dc_dir = './dataset/dc/train' 

In [ ]:
os.mkdir(base_dir)

os.mkdir(train_dir)
os.mkdir(train_dog_dir)
os.mkdir(train_cat_dir)
os.mkdir(test_dir)
os.mkdir(test_dog_dir)
os.mkdir(test_cat_dir)

fnames = ['cat.{}.jpg'.format(i) for i in range(10000)] 
for fname in fnames:
    src = os.path.join('../input/cat-vs-dog/train/train/cat', fname)
    dst = os.path.join(train_cat_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(10000, 11000)] 
for fname in fnames:
    src = os.path.join('../input/cat-vs-dog/train/train/cat', fname)
    dst = os.path.join(test_cat_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(10000)] 
for fname in fnames:
    src = os.path.join('../input/cat-vs-dog/train/train/dog', fname)
    dst = os.path.join(train_dog_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(10000, 11000)] 
for fname in fnames:
    src = os.path.join('../input/cat-vs-dog/train/train/dog', fname)
    dst = os.path.join(test_dog_dir, fname)
    shutil.copyfile(src, dst)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)                        
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

keras内置经典网络实现

In [ ]:
# this will copy the pretrained weights to our kernel


!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/

In [ ]:
from keras import applications

In [ ]:
modelweight = '../input/VGG-19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
model_vgg16 = applications.VGG16(include_top=False, weights='../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
model = keras.Sequential()
model.add(model_vgg16)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model_vgg16.trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=1000,
    epochs=20,
    validation_data=test_generator,
    validation_steps=50)

# 再不用vgg的情况下
# 20000张图片训练  大概能达到0.86    
#  2000张  0.68  